# Pneumonia Detection

In [6]:
import pandas as pd
import numpy as np
import matplotlib as plt
import torch
import pathlib as Path
import os

In [ ]:
# verifica se è presente una GPU per l'esecuzione.
print(f"Is GPU available? {torch.cuda.is_available()}")

Is GPU available? True
GPU Name: NVIDIA GeForce RTX 5050 Laptop GPU


In [10]:
import shutil
import kagglehub

dataset_url = "paultimothymooney/chest-xray-pneumonia"
data_dir = "chest_xray"

def load_data():
    """
    Load original pneumonia images data from the Kaggle's dataset
    """
    # if it finds the data does nothing otherwise it installs it --> cannot pass the dataset to Github for dimension limitations
    if os.path.exists(data_dir):
        print(f"Data found in {data_dir}")
    else:
        # Download latest version --> kagglehub stores the dataset in a sort of cached memory so you need to copy it in local
        path = kagglehub.dataset_download(dataset_url)
        
        shutil.copytree(path, ".", dirs_exist_ok=True)

        print("Path to dataset files:", path)
        
load_data()

Data found in chest_xray
